In [ ]:
# Manage imports
from pathlib import Path
import folium
import AisAudioLabeler as aal
import LabelerUtilities as lu

In [ ]:
# Assign data home directory, collection filename, sampling filepath, and clip home directory
data_home = Path("~").expanduser() / "Data" / "AISonobuoy"
collection_filename = "collection-ais.json"
sampling_filepath = Path("GpxAudioLabeler").parent / "data" / "sampling-ais.json"
clip_home = Path("~").expanduser() / "Datasets" / "AISonobuoy"

In [ ]:
# Load file describing the collection
collection_path = Path(data_home) / collection_filename
collection = lu.load_json_file(collection_path)

In [ ]:
# Load file describing the sampling cases
# NOTE: Currently, the data from this file is unused
sampling = lu.load_json_file(sampling_filepath)

In [ ]:
# For now, assume a single source and hydrophone
if len(collection["sources"]) > 1 or len(collection["hydrophones"]) > 1:
    raise Exception("Only one source and one hydrophone expected")
source = collection["sources"][0]
hydrophone = collection["hydrophones"][0]

In [ ]:
# Load AIS data
ais = aal.get_ais_dataframe(data_home, source)

In [ ]:
# Get hydrophone metadata
hmd = aal.get_hmd_dataframe(data_home, hydrophone)

In [ ]:
# Plot the location of the hydrophone and every nth AIS position
# Note: Modulo 10 still works fairly quickly
# See:
#    https://python-visualization.github.io/folium/quickstart.html
#    https://snyk.io/advisor/npm-package/leaflet
m = folium.Map(location=[hydrophone["lat"], hydrophone["lon"]], zoom_start=13)
folium.Circle(
    radius=0,
    location=[hydrophone["lat"], hydrophone["lon"]],
    popup="hydrophone",
    color="black",
    fill=True,
    ).add_to(m)
divisor = 1000
for index, row in ais.iterrows():
    if index % divisor != 0:
        continue
    folium.Circle(
        radius=0,
        location=[row["lat"], row["lon"]],
        popup=f"{row['mmsi']}\n{row['shiptype']}",
        color="crimson",
        fill=False,
    ).add_to(m)

In [ ]:
# Display the map
m

In [ ]:
# Save the map
m.save(data_home / f"map-{source['name']}-{source['prefix']}-{divisor}.html")